In [ ]:
# https://colab.research.google.com/github/chchae/VSLecture/blob/master/rdkit/RDKit_DeepChem_Solubility.ipynb
#
#! wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
#! chmod +x Miniconda3-latest-Linux-x86_64.sh
# ! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

! wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
! conda install -q -y -c rdkit rdkit 

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
!pip install git+https://github.com/keras-team/keras-tuner.git

In [ ]:
from google.colab import auth auth.authenticate_user()
from google.colab import drive drive.mount('./MyDrive')

driver_name = './MyDrive/My Drive/Colab Notebooks/'

In [ ]:
# driver_name = '../'

In [ ]:
import numpy as np
import tensorflow as tf

import deepchem as dc
from deepchem.models import GraphConvModel
from deepchem.molnet import load_delaney

In [ ]:
delaney_tasks = [ 'measured log solubility in mols per litre' ]

In [ ]:
def prepare_dataset() :
    featurizer = dc.feat.ConvMolFeaturizer()
    loader = dc.data.CSVLoader( tasks=delaney_tasks, feature_field="smiles", featurizer=featurizer )
    dataset_file = driver_name + 'data/delaney-processed.csv'
    dataset = loader.featurize( dataset_file, shard_size=8192 )
    return dataset

In [ ]:
def transform_dataset( dataset ) :
    transformers = [ dc.trans.NormalizationTransformer( transform_y = True, dataset=dataset ) ]
    for transformer in transformers:
        dataset = transformer.transform(dataset)

    splitter = dc.splits.IndexSplitter()
    train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)
    return train_dataset, valid_dataset, test_dataset, transformers

In [ ]:
def make_model( ) :
    metric = dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean)
    batch_size = 128
    model = GraphConvModel( len(delaney_tasks), batch_size=batch_size, mode='regression')
    return model, metric;

In [ ]:
import matplotlib.pyplot as plt

def plot( Y_train, Y_train_pred, Y_test, Y_test_pred ):
    plt.figure( figsize=(11,5) )

    plt.subplot( 1, 2, 1 )
    plt.scatter( x=Y_train, y=Y_train_pred, c="#7CAE00", alpha=0.3 )
    #z = np.polyfit( Y_train, Y_train_pred, 1 )
    #p = np.poly1d(z)
    #plt.plot( Y_train, p(Y_train), "#F8766D" )
    plt.xlabel( 'Experimental LogS' )
    plt.ylabel( 'Predicted LogS' )

    plt.subplot( 1, 2, 2 )
    plt.scatter( x=Y_test, y=Y_test_pred, c="#F8766D", alpha=0.3 )
    #z = np.polyfit( Y_test, Y_test_pred, 1 )
    #p = np.poly1d(z)
    #plt.plot( Y_test, p(Y_test), "#F8766D" )
    plt.xlabel( 'Experimental LogS' )
    plt.ylabel( 'Predicted LogS' )

    plt.show()

In [ ]:
def main():
    dataset = prepare_dataset()
    train_dataset, valid_dataset, test_dataset, transformers = transform_dataset( dataset )
    
    model, metric = make_model()
    result = model.fit(train_dataset, nb_epoch=100)
    print( 'result = %.2f' % result )
    
    
    print("Evaluating model")
    train_scores = model.evaluate(train_dataset, [metric], transformers)
    valid_scores = model.evaluate(valid_dataset, [metric], transformers)

    print("Train scores :", train_scores)
    print("Validation scores : ", valid_scores)
    
    predicted_train = model.predict( train_dataset )
    predicted_valid = model.predict( valid_dataset )

    plot( train_dataset.y, predicted_train, valid_dataset.y, predicted_valid )

In [ ]:
main()

Release GPU Memory

In [ ]:
from numba import cuda

device = cuda.get_current_device()
device.reset()